**1st Part**

In [7]:
import numpy as np
np.set_printoptions(linewidth=300, suppress=True)

#simplex algorithm to solve LP
def simplex_algorithm(tableau, num_vars):

    tableau = np.array(tableau, dtype=float)  #converting tableau to numpy array

    num_rows, num_cols = tableau.shape

    while True:
        # Step 1: Check if optimality is reached (no negative values in the objective function row)
        # The objective function row is the last row of the tableau
        if all(tableau[-1, :-1] >= 0):
            break  # If all values in the last row are >= 0, we are done

        # Step 2: Find entering variable (most negative value in the last row)
        pivot_col = np.argmin(tableau[-1, :-1])  # Column with the most negative coefficient

        # Step 3: Find the leaving variable (minimum positive ratio of RHS to pivot column)
        ratios = []
        for i in range(num_rows - 1):
            if tableau[i, pivot_col] > 0:
                ratios.append(tableau[i, -1] / tableau[i, pivot_col])
            else:
                ratios.append(float('inf'))  # Ignore negative or zero entries

        pivot_row = np.argmin(ratios)  # Row with the minimum positive ratio

        # Step 4: Perform the pivot (normalize pivot row and update other rows)
        pivot_value = tableau[pivot_row, pivot_col]
        tableau[pivot_row, :] /= pivot_value  # Normalize pivot row

        for i in range(num_rows):
            if i != pivot_row:
                row_factor = tableau[i, pivot_col]
                tableau[i, :] -= row_factor * tableau[pivot_row, :]

        # Continue looping until we reach optimality

    # Step 5: Extract the optimal solution
    optimal_values = [0] * num_vars
    for i in range(num_vars):
        # If a column corresponds to a basic variable, it will have exactly one 1 and the rest 0s
        column = tableau[:, i]
        if np.count_nonzero(column[:-1]) == 1 and np.sum(column[:-1]) == 1:
            row_index = np.where(column == 1)[0][0]
            optimal_values[i] = tableau[row_index, -1]

    return optimal_values

Defining artifical variables and formulating intial LP from initial guess (1, 2.5) (as done in notes)

min z1 + z2 + z3 + z4 + z5 + z6  
x1 + x2 - z1 = 3  
x1 - 2x2 + z2 >= -2   
-x1 - 2x2 + z3 >= -6   
-x1 + 2x2 + z4 >= -2   
x1 + z5  >=  0   
x2 + z6 >=  0  

now convert to standard LP and then solve using LP phase method

Phase 1 LP:  
min a1  
x1 + x2 - z1 + a1 = 3  
-x1 + 2x2 - z2 + s1= 2  
x1 + 2x2 - z3 + s2 = 6  
x1 -2x2 - z4 + s3 = 2  
-x1 - z5 + s4 = 0  
-x2 - z6 + s5 = 0  

(similar to what was done in HW7)

In [8]:
tableau = [
  # [x1, x2, z1, z2, z3, z4, z5, z6, s1, s2, s3, s4, s5, a1 b/c]
    [1,  1, -1,  0,  0,  0, 0, 0, 0, 0, 0,  0,  0, 1, 3],
    [-1, 2,  0, -1,  0,  0, 0, 0, 1, 0, 0,  0,  0, 0, 2],
    [1,  2,  0,  0, -1,  0, 0, 0, 0, 1, 0,  0,  0, 0, 6],
    [1, -2,  0,  0,  0, -1, 0, 0, 0, 0, 1,  0,  0, 0, 2],
    [-1,  0,  0,  0,  0,  0, -1, 0, 0, 0, 0, 1,  0, 0, 0],
    [0,  -1,  0,  0,  0,  0, 0, -1, 0, 0, 0,  0, 1, 0, 0],
    [0,  0,  0,  0, 0,  0, 0, 0, 0, 0, 0,  0,  0, 1, 0],
]

tableau = np.array(tableau)

num_vars = 14

#need to make a1 basic
tableau[-1,:] = tableau[-1,:] - tableau[0,:]

print(tableau)


[[ 1  1 -1  0  0  0  0  0  0  0  0  0  0  1  3]
 [-1  2  0 -1  0  0  0  0  1  0  0  0  0  0  2]
 [ 1  2  0  0 -1  0  0  0  0  1  0  0  0  0  6]
 [ 1 -2  0  0  0 -1  0  0  0  0  1  0  0  0  2]
 [-1  0  0  0  0  0 -1  0  0  0  0  1  0  0  0]
 [ 0 -1  0  0  0  0  0 -1  0  0  0  0  1  0  0]
 [-1 -1  1  0  0  0  0  0  0  0  0  0  0  0 -3]]


In [9]:
simplex_algorithm(tableau, num_vars)

[2.6666666666666665,
 0.3333333333333333,
 0,
 0,
 0,
 0,
 0,
 0,
 4.0,
 2.666666666666667,
 0,
 2.6666666666666665,
 0.3333333333333333,
 0]

Here we can see z1, z2...,z6 = 0 , a1 = 0 and intial guess for x1 = 8/3 and x2 = 1/3. So the correct feasible initial guess is (8/3, 1/3). It can be observed that this satisfies all the constraints

In [10]:
#ECQP
def ECQP(G, d, A_k, b_k, x):
  i = A_k.shape[0]
  Z = np.zeros((i,i))
  L1 = np.hstack((G, -A_k.T))
  L2 = np.vstack((L1, np.hstack((A_k, Z))))

  R1 = -(np.dot(G, x) + d)
  R2 = b_k - np.dot(A_k, x)
  R3 = np.vstack((R1, R2))

  result = np.linalg.pinv(L2) @ R3
  n = G.shape[0]

  p = result[:n]
  p = np.round(p, 7)
  lamda = result[n:]

  return p , lamda

#alpha
def min_alpha(A, b, p, x):
  n = len(b)
  alpha_vec = np.zeros(n)
  for i in range(n):
    if np.dot(A[i], p) < 0:
      alpha_vec[i] = (b[i] - np.dot(A[i], x))/np.dot(A[i], p)
    else:
      alpha_vec[i] = np.inf

  alpha_1 = np.min(alpha_vec)
  alpha = min(1, alpha_1)

  return alpha_vec, alpha

In [11]:
A_eq = np.array([1, 1])
b_eq = np.array([3])

A_iq = np.array([[1, -2],
                 [-1, -2],
                 [-1, 2],
                 [1, 0],
                 [0, 1]])

b_iq = np.array([[-2],
                 [-6],
                 [-2],
                 [0],
                 [0]])

x1 = np.array([[8/3],
             [1/3]])

G = np.array([[2, 0],
              [0, 2]])

d = np.array([[-2],
              [-5]])

# Iteration 1, equality and 1st inequality in working set
print('ITERATION 1')
print()
A_k1 = np.vstack((A_eq, A_iq[0]))
b_k1 = np.vstack((b_eq, b_iq[0]))

p1, lamda1 = ECQP(G, d, A_k1, b_k1, x1)

print(f'p1:\n {p1}')
print()
print(f'lamda1:\n {lamda1}')
print()
print(f'p1 norm:\n{np.linalg.norm(p1)}')
print()
##norm of p1 is not zero, need to find alpha
A1 = np.vstack((A_iq[1], A_iq[2], A_iq[3], A_iq[4]))
b1 = np.vstack((b_iq[1], b_iq[2], b_iq[3], b_iq[4]))

alpha_vec1, alpha1 = min_alpha(A1, b1, p1, x1)
print(f'alpha_max: \n{alpha1}')
print()
print(f'alpha_vec1: \n{alpha_vec1}')
print()

x2 = x1 + alpha1*p1

#since alpha_max = 1, no constraint needs to be added
# Iteration 2, equality and 1st and 3rd inequality in working set
print('ITERATION 2')
print()
A_k2 = np.vstack((A_eq, A_iq[0]))
b_k2 = np.vstack((b_eq, b_iq[0]))
p2, lamda2 = ECQP(G, d, A_k2, b_k2, x2)
print(f'p2:\n {p2}')
print()
print(f'lamda2:\n {lamda2}')
print()
print(f'p2 norm:\n{np.linalg.norm(p2)}')
print()

#norm p2 = 0 and lamda > 0 for inequality, optimal solution obtained
A2 = np.vstack((A_iq[1], A_iq[2], A_iq[3], A_iq[4]))
b2 = np.vstack((b_iq[1], b_iq[2], b_iq[3], b_iq[4]))
alpha_vec2, alpha2 = min_alpha(A2, b2, p2, x2)
print(f'alpha_max: \n{alpha2}')
print()
print(f'alpha_vec1: \n{alpha_vec2}')
print()

x3 = x2 + alpha2*p2

print(f'Final answer: \n {x3}')

ITERATION 1

p1:
 [[-1.3333333]
 [ 1.3333333]]

lamda1:
 [[-0.11111111]
 [ 0.77777778]]

p1 norm:
1.8856180360236747

alpha_max: 
1

alpha_vec1: 
[2.00000005        inf 2.00000005        inf]

ITERATION 2

p2:
 [[-0.]
 [ 0.]]

lamda2:
 [[-0.11111111]
 [ 0.77777778]]

p2 norm:
0.0

alpha_max: 
1

alpha_vec1: 
[inf inf inf inf]

Final answer: 
 [[1.33333337]
 [1.66666663]]


<ipython-input-10-89a72bdb1de4>:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  alpha_vec[i] = (b[i] - np.dot(A[i], x))/np.dot(A[i], p)


**Final answer: (1.33, 1.66)**

**2nd Part**

Big M reformulation  
constraints:  
-x1 - x2 + t >= -3  
x1 + x2 + t >= 3  
x1 - 2x2 + t >= -2  
-x1 - 2x2 + t >= -6  
-x1 + 2x2 + t >= -2  
x1 + t >= 0  
x2 + t >= 0  

t = 2 satisfies all for (x1, x2) = (1, 2.5)

In [29]:
A_iq = np.array([[-1, -1 , 1],
                  [1, 1 , 1],
                  [1, -2 , 1],
                  [-1, -2 , 1],
                  [-1, 2 , 1],
                  [1, 0, 1],
                  [0, 1 , 1]])

b_iq = np.array([[-3],
                 [3],
                 [-2],
                 [-6],
                 [-2],
                 [0],
                 [0]])

x1 = np.array([[1],
               [2.5],
               [2]])

G = np.array([[2, 0, 0],
              [0, 2, 0],
              [0, 0, 0]])

d = np.array([[-2],
              [-5],
              [100]])

# Iteration 1, 1st inequality in working set
print('ITERATION 1')
print()
A_k1 = np.array([A_iq[0]])
b_k1 = np.array([b_iq[0]])

p1, lamda1 = ECQP(G, d, A_k1, b_k1, x1)

print(f'p1:\n {p1}')
print()
print(f'lamda1:\n {lamda1}')
print()
print(f'p1 norm:\n{np.linalg.norm(p1)}')
print()

##norm of p1 is not zero, need to find alpha
A1 = np.vstack((A_iq[1], A_iq[2], A_iq[3], A_iq[4], A_iq[5], A_iq[6]))
b1 = np.vstack((b_iq[1], b_iq[2], b_iq[3], b_iq[4], b_iq[5], b_iq[6]))

alpha_vec1, alpha1 = min_alpha(A1, b1, p1, x1)
print(f'alpha_max: \n{alpha1}')
print()
print(f'alpha_vec1: \n{alpha_vec1}')
print()

x2 = x1 + alpha1*p1

# Iteration 2, 1st inequality and 3rd inequality in working set
print('ITERATION 2')
print()
A_k2 = np.vstack((A_iq[0], A_iq[2]))
b_k2 = np.vstack((b_iq[0], b_iq[2]))

p2, lamda2 = ECQP(G, d, A_k2, b_k2, x2)

print(f'p2:\n {p2}')
print()
print(f'lamda2:\n {lamda2}')
print()
print(f'p2 norm:\n{np.linalg.norm(p2)}')
print()

##norm of p2 is not zero, need to find alpha
A2 = np.vstack((A_iq[1], A_iq[3], A_iq[4], A_iq[5], A_iq[6]))
b2 = np.vstack((b_iq[1], b_iq[3], b_iq[4], b_iq[5], b_iq[6]))

alpha_vec2, alpha2 = min_alpha(A2, b2, p2, x2)
print(f'alpha_max: \n{alpha2}')
print()
print(f'alpha_vec1: \n{alpha_vec2}')
print()

x3 = x2 + alpha2*p2

# Iteration 3, 1st inequality, 2nd and 3rd inequality in working set
print('ITERATION 3')
print()
A_k3 = np.vstack((A_iq[0], A_iq[1], A_iq[2]))
b_k3 = np.vstack((b_iq[0], b_iq[1], b_iq[2]))

p3, lamda3 = ECQP(G, d, A_k3, b_k3, x3)

print(f'p3:\n {p3}')
print()
print(f'lamda3:\n {lamda3}')
print()
print(f'p3 norm:\n{np.linalg.norm(p3)}')
print()

##norm of p3 is not zero, need to find alpha
A3 = np.vstack(( A_iq[3], A_iq[4], A_iq[5], A_iq[6]))
b3 = np.vstack(( b_iq[3], b_iq[4], b_iq[5], b_iq[6]))

alpha_vec3, alpha3 = min_alpha(A3, b3, p3, x3)
print(f'alpha_max: \n{alpha3}')
print()
print(f'alpha_vec1: \n{alpha_vec3}')
print()

x4 = x3 + alpha3*p3

# Iteration 4, no change in working set
print('ITERATION 4')
print()
A_k4 = np.vstack((A_iq[0], A_iq[1], A_iq[2]))
b_k4 = np.vstack((b_iq[0], b_iq[1], b_iq[2]))

p4, lamda4 = ECQP(G, d, A_k4, b_k4, x4)

print(f'p4:\n {p4}')
print()
print(f'lamda4:\n {lamda4}')
print()
print(f'p4 norm:\n{np.linalg.norm(p4)}')
print()

print(f'Final answer:\n {x4}')

ITERATION 1

p1:
 [[ -50. ]
 [ -50. ]
 [-101.5]]

lamda1:
 [[100.]]

p1 norm:
123.70226352011511

alpha_max: 
-0.0

alpha_vec1: 
[ 0.01240695 -0.                 inf  0.05280528  0.01980198  0.02970297]

ITERATION 2

p2:
 [[-29.4]
 [-60.3]
 [-91.2]]

lamda2:
 [[79.4]
 [20.6]]

p2 norm:
113.21612076025215

alpha_max: 
0.013819789939192926

alpha_vec1: 
[0.01381979        inf 0.04385965 0.02487562 0.02970297]

ITERATION 3

p3:
 [[ 0.7396352]
 [ 0.       ]
 [-0.7396352]]

lamda3:
 [[49.66666667]
 [49.55555556]
 [ 0.77777778]]

p3 norm:
1.0460021310485368

alpha_max: 
1

alpha_vec1: 
[1.90134518 3.70403566        inf 3.25336304]

ITERATION 4

p4:
 [[-0.]
 [ 0.]
 [ 0.]]

lamda4:
 [[49.66666667]
 [49.55555556]
 [ 0.77777778]]

p4 norm:
0.0

Final answer:
 [[ 1.33333338]
 [ 1.66666667]
 [-0.00000004]]


<ipython-input-10-89a72bdb1de4>:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  alpha_vec[i] = (b[i] - np.dot(A[i], x))/np.dot(A[i], p)


**Final answer: (1.33, 1.66)**